In [3]:
import random
import jieba
import gensim

class ProcessDocument:
    def __init__(self, documents,stopwords_path):
        self.documents = documents
        self.words = {}
        self.stopwords = set()
        self.load_stopwords(stopwords_path)
    def load_stopwords(self, path):
        with open(path, 'r',encoding='utf-8') as fin:
            for line in fin:
                self.stopwords.add(line.strip())
                
    def is_chn(self, words):
        for w in words:
            if not (u'\u4e00' <= w <= u'\u9fff' or \
               u'a' <= w <= u'z' or u'A' <= w <= u'Z'or \
               u'0' <= w <= u'9'):
                return False
        return True
    
    def process(self):
        texts = []
        for document in self.documents:
            text = []
            document = document.split('\t')[-1]
            for word in jieba.cut(document.strip()):
                if not self.is_chn(word): continue
                if word in self.stopwords: continue
                if len(word) < 2: continue
                if word not in self.words:
                    self.words[word] = 0
                self.words[word] += 1
                text.append(word)
            texts.append(text)
            
        self.words = sorted(self.words.items(), key=lambda x:x[1], reverse=True)
        self.words = [word for word,count in self.words]
        return self.words,texts
    
    def get_word2id(self):
        return dict([(word,i) for i,word in enumerate(self.words)])
    def get_id2word(self):
        return dict(enumerate(self.words))
    
documents = []
with open('3500.txt',encoding="utf-8") as fin:
    for line in fin:
        documents.append(line)
pd = ProcessDocument(documents, 'stopwords.txt')
words,texts = pd.process()
word2id = pd.get_word2id()
id2word = pd.get_id2word()
print(len(word2id))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yuwan\AppData\Local\Temp\jieba.cache
Loading model cost 0.581 seconds.
Prefix dict has been built successfully.


13917


In [4]:
class LDAModel:
    def __init__(self,num_topic, word2id,id2word,documents):
        self.K = num_topic
        self.alpha = 50/self.K
        self.beta = 0.01
        self.z = []
        self.word2id = word2id
        self.id2word = id2word
        self.documents = documents
        
    def init_topic(self, n_mk, n_m, n_kt, n_k):
        M = len(self.documents)
        V = len(self.word2id)
        for m in range(M):
            self.z.append([])
            for w in self.documents[m]:
                k = random.randint(0,self.K-1)
                t = word2id[w]
                self.z[m].append(k)
                n_mk[m][k] += 1
                n_m[m] += 1
                n_kt[k][t] += 1
                n_k[k] += 1 
                
    def reSampling(self, m, t, n_mk, n_m, n_kt, n_k):
        
        topic_alpha = self.alpha * self.K
        topic_beta = self.beta * len(self.word2id)
        
        p = [0] * self.K
        for k in range(self.K):
            p[k] = (n_kt[k][t] + self.beta)/(n_k[k] + topic_beta) * (n_mk[m][k] + self.alpha)/(n_m[m] + topic_alpha)
            if k > 1:
                p[k] += p[k-1]
        u = random.random() * p[self.K -1]
        
        new_k = 0
        for i in range(self.K):
            if p[i] >= u:
                new_k = i
                break
        return new_k
    
    def calc_theta(self, n_mk, n_m):
        topic_alpha = self.alpha * self.K
        theta = [[0 for i in range(self.K)] for i in range(len(self.documents))]
        for m in range(len(self.documents)):
            for k in range(self.K):
                theta[m][k] = (n_mk[m][k] + self.alpha) / (n_m[m] + topic_alpha)
        return theta
    
    def calc_phi(self, n_kt, n_k):
        topic_beta = self.beta * len(self.word2id)
        phi = [[0 for i in range(len(self.word2id))] for i in range(self.K)]
        for k in range(self.K):
            for t in range(len(self.word2id)):
                phi[k][t] = (n_kt[k][t] + self.beta) / (n_k[k] + topic_beta)
        return phi
    
    def lda(self, n_mk, n_m, n_kt, n_k, times):
        print('run lda')
        M = len(self.documents)
        for epoch in range(times):
            #print('epoch %d' % epoch)
            for m in range(M):
                for i in range(len(self.documents[m])):
                    k = self.z[m][i]
                    word = self.documents[m][i]
                    t = word2id[word]
                    n_mk[m][k] -= 1
                    n_m[m] -= 1
                    n_kt[k][t] -= 1
                    n_k[k] -= 1

                    new_k = self.reSampling(m, t, n_mk, n_m, n_kt, n_k)
                    n_mk[m][new_k] += 1
                    n_m[m] += 1
                    n_kt[new_k][t] += 1
                    n_k[new_k] += 1
                    self.z[m][i] = new_k
                
        theta = self.calc_theta(n_mk, n_m)
        phi = self.calc_phi(n_kt, n_k)
        return theta, phi
    
    def run_lda(self, times):
        n_mk = [[0 for i in range(self.K)] for i in range(len(self.documents))]
        n_m = [0 for i in range(len(self.documents))]
        n_kt = [[0 for i in range(len(words))] for i in range(len(self.documents))]
        n_k = [0 for i in range(self.K)]
        
        self.init_topic(n_mk, n_m, n_kt, n_k)
        theta,phi = self.lda(n_mk, n_m, n_kt, n_k, times)
        return theta,phi
    
    def get_topic_N(self,phi,N):
        topics_dic = {}
        for k in range(len(phi)):
            dic = {}
            for i in range(len(phi[k])):
                dic[i] = phi[k][i]
            word_prob_list = sorted(dic.items(), key = lambda d:d[1], reverse = True)
            topics_dic[k] = word_prob_list
            print('topic %d' % k, end='\t')
            for word_pro in word_prob_list[0:N]:
                print(self.id2word[word_pro[0]],word_pro[1], end=' ')
            print('\n')
        return topics_dic
    
    def get_doc_topic(self, theta, topics_dic):
        #doc_dic = {}
        for m in range(len(theta)):
            dic = {}
            for i in range(len(theta[m])):
                dic[i] = theta[m][i]
            topic_prob_list = sorted(dic.items(), key = lambda d:d[1], reverse=True)
            #doc_dic['doc ' + str(m)] = topic_prob_list
            print('doc %d ' % m, end = '\t')
            for top_prob in topic_prob_list[:1]:
                #print(topics_dic[top_prob[0]])
                for word_pro in topics_dic[top_prob[0]][0:15]:
                    print(self.id2word[word_pro[0]],word_pro[1], end=' ')
                print('')

In [26]:
lda = LDAModel(15, word2id, id2word, texts)
theta, phi = lda.run_lda(300)
top_dic = lda.get_topic_N(phi,50)
lda.get_doc_topic(theta, top_dic)

run lda
topic 0	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 媒体 0.003949527732160981 永远 0.00378028643525364 加盟 0.0037238726696178595 逐梦 0.0034982176070747378 日本 0.0034982176070747378 喝彩 0.003328976310167397 技术 0.003216148778895836 急需 0.0031033212476242753 举办 0.0031033212476242753 花滑 0.002990493716352715 意大利 0.0029340799507169345 春天 0.002877666185081154 虎年 0.002821252419445374 圆梦 0.0027648386538095936 传递 0.002652011122538033 一场 0.002652011122538033 享受 0.0025391835912664724 浪漫 0.002369942294359131 大使 0.002369942294359131 表演 0.002369942294359131 朋友 0.002369942294359131 20 0.002369942294359131 舍不得 0.002369942294359131 张真源 0.002369942294359131 热爱 0.0022007009974517903 我国 0.0020878734

doc 37 	冬奥 0.19494260977541125 视频 0.1378549236156236 微博 0.13522415466816795 哈哈哈 0.03762262671756327 喜欢 0.025258012664521715 刘雨昕 0.020785705453847107 音乐 0.012367244821989018 邓超 0.011578014137752323 指路 0.010525706558770062 主题 0.010262629664024497 宣传 0.009473398979787802 演员 0.00842109140080554 如约 0.00842109140080554 哈哈哈哈 0.00842109140080554 祝福 0.007631860716568846 
doc 38 	真的 0.099270562371205 冰墩 0.05821208522492509 不行 0.053274673416195226 感觉 0.022091019887375038 可爱 0.017153608078645174 米兰 0.012995787608135814 工作 0.012995787608135814 开心 0.01273592382872898 晚上 0.012476060049322144 节目 0.011436604931694804 一点 0.0106570135934743 明天 0.009357694696440126 好像 0.009357694696440126 好多 0.00909783091703329 带来 0.008318239578812786 
doc 39 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0

doc 140 	真的 0.099270562371205 冰墩 0.05821208522492509 不行 0.053274673416195226 感觉 0.022091019887375038 可爱 0.017153608078645174 米兰 0.012995787608135814 工作 0.012995787608135814 开心 0.01273592382872898 晚上 0.012476060049322144 节目 0.011436604931694804 一点 0.0106570135934743 明天 0.009357694696440126 好像 0.009357694696440126 好多 0.00909783091703329 带来 0.008318239578812786 
doc 141 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 142 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685

doc 234 	冬奥 0.15137190145082388 疫情 0.06368029529136547 再见 0.024016707274625827 成功 0.01969958204831403 防控 0.017810839761802616 北京 0.016461738128580178 盛会 0.014842816168713253 新冠 0.014572995842068765 精神 0.013763534862135303 火炬手 0.013763534862135303 感谢 0.013223894208846329 中心 0.01079551126904594 谢谢 0.009986050289112479 鸟巢 0.009716229962467992 城市 0.009716229962467992 
doc 235 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 236 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.0047957342166

doc 344 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 345 	中国 0.13977069986976715 科技 0.04140601279242792 冬奥 0.03362717274294447 美国 0.03362717274294447 冰球 0.02986321788029118 闪耀 0.027855775286876095 男子 0.025597402369284122 支持 0.024844611396753466 精彩 0.024091820424222806 拼过 0.016062050050562462 徽章 0.014054607457147374 中国队 0.013301816484616716 红包 0.013301816484616716 观众 0.01129437389120163 vs 0.0100397222703172 
doc 346 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.00479573421669768

...

doc 2916 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 2917 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 2918 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734

doc 3021 	肖战 0.1802021494752303 加油 0.08670345829112906 健儿 0.08090123454743484 手绘 0.059184339963893585 少年 0.04774567029775354 雪容 0.04128033526906569 融为 0.04094877962656888 团肖战 0.03796477884409756 快乐 0.03315722202789378 DAYTOY 0.027520776105447956 元宵节 0.024702553144225044 雪容融 0.023376330574237793 电影 0.021055441076760106 喜欢 0.01989499632802126 报道 0.019563440685524448 
doc 3022 	冬奥 0.19494260977541125 视频 0.1378549236156236 微博 0.13522415466816795 哈哈哈 0.03762262671756327 喜欢 0.025258012664521715 刘雨昕 0.020785705453847107 音乐 0.012367244821989018 邓超 0.011578014137752323 指路 0.010525706558770062 主题 0.010262629664024497 宣传 0.009473398979787802 演员 0.00842109140080554 如约 0.00842109140080554 哈哈哈哈 0.00842109140080554 祝福 0.007631860716568846 
doc 3023 	冬奥 0.15137190145082388 疫情 0.06368029529136547 再见 0.024016707274625827 成功 0.01969958204831403 防控 0.017810839761802616 北京 0.016461738128580178 盛会 0.014842816168713253 新冠 0.014572995842068765 精神 0.013763534862135303 火炬手 0.013763534862135303 感谢 0.013223894208

doc 3125 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 3126 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 3127 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734

doc 3234 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 3235 	冬奥 0.400590543040442 健儿 0.04990391958468999 今日 0.03005373335134554 时代 0.018198761017542604 星运 0.015441790707355872 遗憾 0.014614699614299852 谷爱凌 0.01351191149022516 两个 0.01075494118003843 之约 0.009927850086982413 冬天 0.007997970869851703 万物 0.007997970869851703 骄傲 0.007170879776795684 漂亮 0.005792394621702319 错过 0.005516697590683646 鞠婧 0.004689606497627627 
doc 3236 	北京 0.3218592983471749 冬奥会 0.20509033514176792 2022 0.05888173926381127 助力 0.023504701682512116 火炬 0.018309472387356297 呜呜 0.011877283736210996 金博洋 0.008413797539440448 欢乐 0.008166405668242551 网络 0.007424230054648865 张艺谋 0.006682054441055176 落幕 0.0061872706

doc 3336 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 3337 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734216697685 文化 0.004682906685426125 物资 0.004231596560339882 成绩 0.004118769029068322 期间 0.004005941497796761 
doc 3338 	比赛 0.01455531567168768 开幕 0.013652695421515195 立春 0.012411592577528028 闭幕式 0.011734627389898665 羽生 0.00919600793628855 选手 0.007842077561029823 现场 0.00750359496721514 结弦 0.007277939904672019 推广 0.005359871873055489 垃圾 0.005190630576148148 时间 0.004795734

In [22]:
from gensim import corpora, models

In [23]:
dictionary = corpora.Dictionary(texts)

In [24]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [25]:
texts_tf_idf = models.TfidfModel(corpus)[corpus]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=9,update_every=0,passes=20)
texts_lda = lda[texts_tf_idf]
res_dic = dict(lda.print_topics(num_topics=3, num_words=20))
print(res_dic)
print('-----------')
count = 0
for doc1 in texts_lda:
    #for doc in doc1:
    print('doc %d ' % count, res_dic[doc1[0][0]])
    count +=1
    print(count, '------------------')

{8: '0.042*"冬奥" + 0.013*"周深" + 0.009*"梦想" + 0.007*"指路" + 0.006*"中国" + 0.006*"疫情" + 0.005*"科技" + 0.005*"火炬手" + 0.005*"奥运" + 0.004*"北京" + 0.004*"卡布" + 0.004*"徐坤" + 0.004*"主题" + 0.004*"黄明昊" + 0.004*"滑雪" + 0.003*"视频" + 0.003*"链接" + 0.003*"音乐" + 0.003*"运动员" + 0.003*"官方"', 7: '0.658*"刘宇" + 0.046*"国风" + 0.045*"美少年" + 0.006*"冬奥" + 0.003*"星运" + 0.003*"电台" + 0.002*"加盟" + 0.002*"INTO1" + 0.001*"冰雪" + 0.001*"加油" + 0.001*"运动" + 0.001*"喝彩" + 0.001*"播报" + 0.001*"ly" + 0.001*"健儿" + 0.000*"nc" + 0.000*"香港" + 0.000*"一道" + 0.000*"助威" + 0.000*"报道"', 2: '0.073*"冬奥" + 0.042*"北京" + 0.024*"冬奥会" + 0.014*"疫情" + 0.011*"中国" + 0.010*"运动员" + 0.009*"开幕式" + 0.008*"开幕" + 0.007*"视频" + 0.007*"立春" + 0.007*"微博" + 0.007*"羽生" + 0.007*"2022" + 0.006*"闭幕式" + 0.006*"结弦" + 0.005*"期待" + 0.005*"结束" + 0.005*"美国" + 0.004*"选手" + 0.004*"比赛"'}
-----------


KeyError: 0